In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.metrics import AUC, Precision
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.applications.inception_v3 import InceptionV3

Data obtenida de: https://ieee-dataport.org/open-access/retinal-fundus-multi-disease-image-dataset-rfmid
Un paper explicando que significa cada codigo de enfermedad puede encontrarse en: https://www.researchgate.net/publication/348994714_Retinal_Fundus_Multi-Disease_Image_Dataset_RFMiD_A_Dataset_for_Multi-Disease_Detection_Research/link/601a5f5c92851c4ed545fb2d/download

In [2]:
training_raw = pd.read_csv("B. RFMiD_Challenge_Dataset/2. Groundtruths/a. RFMiD_Training_Labels.csv")
print(training_raw.shape)
training_raw.head(10)

(1920, 30)


,ID,Disease_Risk,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,AION,PT,RT,RS,CRS,EDN,RPEC,MHL,RP,OTHER
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


La columna de Disease_Risk nos sirve para determinar si una imagen representa un paciente con algun padecimiento, pero tenemos un vector esparso que nos dice que padecimiento es, por lo cual la columna es superflua, sin embargo nos puede servir para determinar si un paciente es saludable al invertir su valor de verdad.

Por otra parte, la columna ID nos sera util para determinar el filename correspondiente a cada registro en el dataframe

In [3]:
training = training_raw.copy()
training.rename(columns = {"Disease_Risk": "Healthy"}, inplace = True)
training["Healthy"] = training_raw["Disease_Risk"].apply(lambda x: 1*(x==0))
training["ID"] = training_raw["ID"].astype(str) + ".png"
training.head(10)

,ID,Healthy,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,AION,PT,RT,RS,CRS,EDN,RPEC,MHL,RP,OTHER
0,1.png,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2.png,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.png,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.png,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5.png,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6.png,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7.png,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8.png,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9.png,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,10.png,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Podemos repetir este proceso con los conjuntos de validacion y de prueba

In [4]:
validation_raw = pd.read_csv("B. RFMiD_Challenge_Dataset/2. Groundtruths/b. RFMiD_Validation_Labels.csv")
print(validation_raw.shape)
validation = validation_raw.copy()
validation.rename(columns = {"Disease_Risk": "Healthy"}, inplace = True)
validation["Healthy"] = validation_raw["Disease_Risk"].apply(lambda x: 1*(x==0))
validation["ID"] = validation_raw["ID"].astype(str) + ".png"
validation.head(10)

(640, 30)


,ID,Healthy,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,AION,PT,RT,RS,CRS,EDN,RPEC,MHL,RP,OTHER
0,1.png,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
testing_raw = pd.read_csv("B. RFMiD_Challenge_Dataset/2. Groundtruths/c. RFMiD_Testing_Labels.csv")
print(testing_raw.shape)
testing = validation_raw.copy()
testing.rename(columns = {"Disease_Risk": "Healthy"}, inplace = True)
testing["Healthy"] = testing_raw["Disease_Risk"].apply(lambda x: 1*(x==0))
testing["ID"] = testing_raw["ID"].astype(str) + ".png"
testing.head(10)

(640, 30)


,ID,Healthy,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,AION,PT,RT,RS,CRS,EDN,RPEC,MHL,RP,OTHER
0,1.png,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10.png,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Se debe mencionar que algunos registros son multiclase, por lo cual no podemos usar softmax como nuestra funcion de activacion final

In [6]:
training.iloc[:, 1:].sum(axis=1)

0       1
1       1
2       1
3       2
4       2
       ..
1915    1
1916    2
1917    1
1918    1
1919    1
Length: 1920, dtype: int64

In [7]:
training.iloc[3, :]

ID         4.png
Healthy        0
DR             0
ARMD           0
MH             1
DN             0
MYA            0
BRVO           0
TSLN           0
ERM            0
LS             0
MS             0
CSR            0
ODC            1
CRVO           0
TV             0
AH             0
ODP            0
ODE            0
ST             0
AION           0
PT             0
RT             0
RS             0
CRS            0
EDN            0
RPEC           0
MHL            0
RP             0
OTHER          0
Name: 3, dtype: object

Necesitamos asociar los labels a las imagenes

In [8]:
base_dir = "B. RFMiD_Challenge_Dataset/1. Original Images"
training_dir = "a. Training Set"
validation_dir = "b. Validation Set"
testing_dir = "c. Testing Set"
label_cols = training.columns.values.tolist()[1:]
target_size = (712, 1072)
input_shape = (target_size[0], target_size[1], 3)

train_datagen=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=False,
    rotation_range=90
)

test_datagen = ImageDataGenerator(rescale=1./255)

print("Training:")

train_gen = train_datagen.flow_from_dataframe(
    dataframe = training,
    directory = base_dir+"/"+training_dir+"/",
    x_col = "ID",
    y_col = label_cols,
    target_size = target_size,
    class_mode='raw',
    seed = 1337,
    batch_size = 16
)

print("\nValidation:")

val_gen = test_datagen.flow_from_dataframe(
    dataframe = validation,
    directory = base_dir+"/"+validation_dir+"/",
    x_col = "ID",
    y_col = label_cols,
    target_size = target_size,
    class_mode='raw',
    seed = 1337,
    batch_size = 8
)

print("\nTesting:")

test_gen = test_datagen.flow_from_dataframe(
    dataframe = testing,
    directory = base_dir+"/"+testing_dir+"/",
    x_col = "ID",
    y_col = label_cols,
    target_size = target_size,
    class_mode='raw',
    seed = 1337,
    batch_size = 8
)

Training:
Found 1920 validated image filenames.

Validation:
Found 640 validated image filenames.

Testing:
Found 640 validated image filenames.


# Definiendo modelo

In [13]:
tf.keras.backend.clear_session()

cnn = tf.keras.models.Sequential()

cnn.add(Conv2D(filters=16, kernel_size=(3, 3), padding="same" , activation="relu", input_shape=input_shape))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same" , activation="relu", input_shape=input_shape))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same" , activation="relu", input_shape=input_shape))
cnn.add(BatchNormalization())
cnn.add(MaxPooling2D(pool_size=(2, 2)))

cnn.add(Flatten())
cnn.add(Dense(units = 128, activation = "relu"))
cnn.add(BatchNormalization())

cnn.add(Dense(units = len(label_cols), activation = "sigmoid"))

auc = tf.keras.metrics.AUC(multi_label=True, thresholds=[0,0.7])

cnn.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy", Precision(), auc]
)

cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 712, 1072, 16)     448       
                                                                 
 batch_normalization (BatchN  (None, 712, 1072, 16)    64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 356, 536, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 356, 536, 32)      4640      
                                                                 
 batch_normalization_1 (Batc  (None, 356, 536, 32)     128       
 hNormalization)                                                 
                                                        

In [14]:
history = cnn.fit(
    train_gen,
    validation_data=val_gen,
    epochs = 1)

120/120 [==============================] - 1231s 10s/step - loss: 0.6181 - accuracy: 0.1406 - precision: 0.0672 - auc: 0.4971 - val_loss: 0.4227 - val_accuracy: 0.0281 - val_precision: 0.0689 - val_auc: 0.5000


AttributeError: 'Sequential' object has no attribute 'eval'

In [15]:
cnn.evaluate(test_gen)

80/80 [==============================] - 129s 2s/step - loss: 0.4204 - accuracy: 0.0281 - precision: 0.0733 - auc: 0.5000


[0.4204334616661072, 0.02812499925494194, 0.07334273308515549, 0.5]

In [16]:
cnn.save("fundus_resulting_modesl.h5")

# Prediciendo menos enfermedades

In [24]:
training.iloc[:, 1:].sum(axis = 0).sort_values(ascending = False)

Healthy    401
DR         376
MH         317
ODC        282
TSLN       186
DN         138
MYA        101
ARMD       100
BRVO        73
ODP         65
ODE         58
LS          47
RS          43
CSR         37
OTHER       34
CRS         32
CRVO        28
RPEC        22
AION        17
AH          16
MS          15
EDN         15
RT          14
ERM         14
PT          11
MHL         11
RP           6
TV           6
ST           5
dtype: int64

Nos quedaremos solo con aquellas enfermedades con al menos 100 ejemplos positivos

In [39]:
training_less = training[["ID", "Healthy", "DR", "MH", "ODC", "TSLN", "DN", "MYA", "ARMD"]]
training_less["OTHER"] = (training_less.iloc[:, 1:].sum(axis = 1)==0)*1
training_less.head(10)

C:\Users\yanco\AppData\Local\Temp\ipykernel_13912\660561726.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_less["OTHER"] = (training_less.iloc[:, 1:].sum(axis = 1)==0)*1


,ID,Healthy,DR,MH,ODC,TSLN,DN,MYA,ARMD,OTHER
0,1.png,0,1,0,0,0,0,0,0,0
1,2.png,0,1,0,0,0,0,0,0,0
2,3.png,0,1,0,0,0,0,0,0,0
3,4.png,0,0,1,1,0,0,0,0,0
4,5.png,0,1,0,0,0,0,0,0,0
5,6.png,0,0,0,0,0,0,1,1,0
6,7.png,0,0,0,0,0,0,1,1,0
7,8.png,0,0,0,0,0,0,1,1,0
8,9.png,0,0,0,0,0,0,0,0,1
9,10.png,1,0,0,0,0,0,0,0,0


In [40]:
validation_less = validation[["ID", "Healthy", "DR", "MH", "ODC", "TSLN", "DN", "MYA", "ARMD"]]
validation_less["OTHER"] = (validation_less.iloc[:, 1:].sum(axis = 1)==0)*1
validation_less.head(10)

C:\Users\yanco\AppData\Local\Temp\ipykernel_13912\4154515618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_less["OTHER"] = (validation_less.iloc[:, 1:].sum(axis = 1)==0)*1


,ID,Healthy,DR,MH,ODC,TSLN,DN,MYA,ARMD,OTHER
0,1.png,0,1,0,0,1,0,0,0,0
1,2.png,0,0,0,0,0,0,0,0,1
2,3.png,0,0,0,0,0,0,0,0,1
3,4.png,0,0,0,0,0,0,0,0,1
4,5.png,0,0,0,0,0,0,0,0,1
5,6.png,0,0,0,0,0,0,0,0,1
6,7.png,0,0,0,0,0,0,0,0,1
7,8.png,0,0,0,0,0,0,0,0,1
8,9.png,0,0,0,0,0,0,0,0,1
9,10.png,0,0,0,0,0,0,0,0,1


In [41]:
testing_less = testing[["ID", "Healthy", "DR", "MH", "ODC", "TSLN", "DN", "MYA", "ARMD"]]
testing_less["OTHER"] = (testing_less.iloc[:, 1:].sum(axis = 1)==0)*1
testing_less.head(10)

C:\Users\yanco\AppData\Local\Temp\ipykernel_13912\3467796201.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testing_less["OTHER"] = (testing_less.iloc[:, 1:].sum(axis = 1)==0)*1


,ID,Healthy,DR,MH,ODC,TSLN,DN,MYA,ARMD,OTHER
0,1.png,0,1,0,0,1,0,0,0,0
1,2.png,0,0,0,0,0,0,0,0,1
2,3.png,0,0,0,0,0,0,0,0,1
3,4.png,0,0,0,0,0,0,0,0,1
4,5.png,0,0,0,0,0,0,0,0,1
5,6.png,0,0,0,0,0,0,0,0,1
6,7.png,0,0,0,0,0,0,0,0,1
7,8.png,0,0,0,0,0,0,0,0,1
8,9.png,0,0,0,0,0,0,0,0,1
9,10.png,0,0,0,0,0,0,0,0,1


In [42]:
label_less_cols = training_less.columns.values.tolist()[1:]
target_size = (712, 1072)
input_shape = (target_size[0], target_size[1], 3)

train_less_gen = train_datagen.flow_from_dataframe(
    dataframe = training_less,
    directory = base_dir+"/"+training_dir+"/",
    x_col = "ID",
    y_col = label_less_cols,
    target_size = target_size,
    class_mode='raw',
    seed = 1337,
    batch_size = 16
)

print("\nValidation:")

val_less_gen = test_datagen.flow_from_dataframe(
    dataframe = validation_less,
    directory = base_dir+"/"+validation_dir+"/",
    x_col = "ID",
    y_col = label_less_cols,
    target_size = target_size,
    class_mode='raw',
    seed = 1337,
    batch_size = 8
)

print("\nTesting:")

test_less_gen = test_datagen.flow_from_dataframe(
    dataframe = testing_less,
    directory = base_dir+"/"+testing_dir+"/",
    x_col = "ID",
    y_col = label_less_cols,
    target_size = target_size,
    class_mode='raw',
    seed = 1337,
    batch_size = 8
)

Found 1920 validated image filenames.

Validation:
Found 640 validated image filenames.

Testing:
Found 640 validated image filenames.


In [43]:
tf.keras.backend.clear_session()

cnn_less = tf.keras.models.Sequential()

cnn_less.add(Conv2D(filters=16, kernel_size=(3, 3), padding="same" , activation="relu", input_shape=input_shape))
cnn_less.add(BatchNormalization())
cnn_less.add(MaxPooling2D(pool_size=(2, 2)))

cnn_less.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same" , activation="relu"))
cnn_less.add(BatchNormalization())
cnn_less.add(MaxPooling2D(pool_size=(2, 2)))

cnn_less.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same" , activation="relu"))
cnn_less.add(BatchNormalization())
cnn_less.add(MaxPooling2D(pool_size=(2, 2)))

cnn_less.add(Flatten())
cnn_less.add(Dense(units = 128, activation = "relu"))
cnn_less.add(BatchNormalization())

cnn_less.add(Dense(units = len(label_less_cols), activation = "sigmoid"))

auc = tf.keras.metrics.AUC(multi_label=True, thresholds=[0,0.7])

cnn_less.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy", Precision(), auc]
)

cnn_less.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 712, 1072, 16)     448       
                                                                 
 batch_normalization (BatchN  (None, 712, 1072, 16)    64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 356, 536, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 356, 536, 32)      4640      
                                                                 
 batch_normalization_1 (Batc  (None, 356, 536, 32)     128       
 hNormalization)                                                 
                                                        

In [44]:
history_less = cnn_less.fit(
    train_less_gen,
    validation_data=val_less_gen,
    epochs = 1)

120/120 [==============================] - 1034s 9s/step - loss: 0.6655 - accuracy: 0.2250 - precision: 0.1759 - auc: 0.5088 - val_loss: 0.4673 - val_accuracy: 0.2125 - val_precision: 0.2119 - val_auc: 0.5000


In [45]:
cnn_less.evaluate(test_less_gen)

80/80 [==============================] - 117s 1s/step - loss: 0.4654 - accuracy: 0.3219 - precision: 0.3213 - auc: 0.5000


[0.46541348099708557, 0.3218750059604645, 0.32131659984588623, 0.5]

In [81]:
cnn_less.save("fundus_resulting_model_less_vars.h5")

Se logra ver un incremento sustancial en los resultados al reducir la cantidad de variables a predecir

# Transfer Learning
Utilizando Inception3

In [73]:
tf.keras.backend.clear_session()
inception = InceptionV3(input_shape = input_shape, 
                        include_top = False, 
                        weights = None,
                        classes = label_less_cols,
                        classifier_activation="sigmoid")

for layer in inception.layers:
    layer.trainable = False
last_layer = inception.get_layer('mixed7')
last_output = last_layer.output

input_1
conv2d
batch_normalization
activation
conv2d_1
batch_normalization_1
activation_1
conv2d_2
batch_normalization_2
activation_2
max_pooling2d
conv2d_3
batch_normalization_3
activation_3
conv2d_4
batch_normalization_4
activation_4
max_pooling2d_1
conv2d_8
batch_normalization_8
activation_8
conv2d_6
conv2d_9
batch_normalization_6
batch_normalization_9
activation_6
activation_9
average_pooling2d
conv2d_5
conv2d_7
conv2d_10
conv2d_11
batch_normalization_5
batch_normalization_7
batch_normalization_10
batch_normalization_11
activation_5
activation_7
activation_10
activation_11
mixed0
conv2d_15
batch_normalization_15
activation_15
conv2d_13
conv2d_16
batch_normalization_13
batch_normalization_16
activation_13
activation_16
average_pooling2d_1
conv2d_12
conv2d_14
conv2d_17
conv2d_18
batch_normalization_12
batch_normalization_14
batch_normalization_17
batch_normalization_18
activation_12
activation_14
activation_17
activation_18
mixed1
conv2d_22
batch_normalization_22
activation_22
conv2d

Utilizando Inception3 como tal, sin agregar mas capas de convolucion

In [74]:
x = Flatten()(last_output)
x = Dense(units = 128, activation = "relu")(x)
x = BatchNormalization()(x)

output = Dense(units = len(label_less_cols), activation = "sigmoid")(x)

base_inception = tf.keras.Model(inception.input, output)

base_inception.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy", Precision(), auc]
)

base_inception.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 712, 1072,   0           []                               
                                3)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 355, 535, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 355, 535, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 conv2d_7 (Conv2D)              (None, 86, 131, 64)  76800       ['activation_6[0][0]']           
                                                                                                  
 conv2d_10 (Conv2D)             (None, 86, 131, 96)  82944       ['activation_9[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 86, 131, 32)  6144        ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 86, 131, 64)  192        ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 86, 131, 64)  192        ['conv2d_7[0][0]']               
 rmalizati

 mixed1 (Concatenate)           (None, 86, 131, 288  0           ['activation_12[0][0]',          
                                )                                 'activation_14[0][0]',          
                                                                  'activation_17[0][0]',          
                                                                  'activation_18[0][0]']          
                                                                                                  
 conv2d_22 (Conv2D)             (None, 86, 131, 64)  18432       ['mixed1[0][0]']                 
                                                                                                  
 batch_normalization_22 (BatchN  (None, 86, 131, 64)  192        ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

 batch_normalization_29 (BatchN  (None, 42, 65, 96)  288         ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_26 (Activation)     (None, 42, 65, 384)  0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 42, 65, 96)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 42, 65, 288)  0          ['mixed2[0][0]']                 
                                                                                                  
 mixed3 (Concatenate)           (None, 42, 65, 768)  0           ['activation_26[0][0]',          
          

                                                                                                  
 activation_39 (Activation)     (None, 42, 65, 192)  0           ['batch_normalization_39[0][0]'] 
                                                                                                  
 mixed4 (Concatenate)           (None, 42, 65, 768)  0           ['activation_30[0][0]',          
                                                                  'activation_33[0][0]',          
                                                                  'activation_38[0][0]',          
                                                                  'activation_39[0][0]']          
                                                                                                  
 conv2d_44 (Conv2D)             (None, 42, 65, 160)  122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_nor

                                                                  'activation_48[0][0]',          
                                                                  'activation_49[0][0]']          
                                                                                                  
 conv2d_54 (Conv2D)             (None, 42, 65, 160)  122880      ['mixed5[0][0]']                 
                                                                                                  
 batch_normalization_54 (BatchN  (None, 42, 65, 160)  480        ['conv2d_54[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_54 (Activation)     (None, 42, 65, 160)  0           ['batch_normalization_54[0][0]'] 
                                                                                                  
 conv2d_55

 batch_normalization_64 (BatchN  (None, 42, 65, 192)  576        ['conv2d_64[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_64 (Activation)     (None, 42, 65, 192)  0           ['batch_normalization_64[0][0]'] 
                                                                                                  
 conv2d_65 (Conv2D)             (None, 42, 65, 192)  258048      ['activation_64[0][0]']          
                                                                                                  
 batch_normalization_65 (BatchN  (None, 42, 65, 192)  576        ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

 dense_1 (Dense)                (None, 9)            1161        ['batch_normalization_94[0][0]'] 
                                                                                                  
Total params: 277,346,985
Trainable params: 268,371,465
Non-trainable params: 8,975,520
__________________________________________________________________________________________________


In [76]:
base_inception_history = base_inception.fit(
    train_less_gen,
    validation_data=val_less_gen,
    epochs = 1)

base_inception.evaluate(test_less_gen)

80/80 [==============================] - 450s 6s/step - loss: 0.4250 - accuracy: 0.2094 - precision: 0.0870 - auc: 0.5000


[0.42500200867652893, 0.20937499403953552, 0.08695652335882187, 0.5]

Se alcanza una buena accuracy, aunque una relativamente pobre precision, al usar InceptionV3 as is, agregar capas adicionales de  convolucion podrian marcar una gran diferencia

In [78]:
tf.keras.backend.clear_session()
inception_trainable = InceptionV3(input_shape = input_shape, 
                        include_top = False, 
                        weights = None,
                        classes = label_less_cols,
                        classifier_activation="sigmoid")

for i, layer in enumerate(inception.layers):
    if i > 16:
        layer.trainable = False
last_layer_trainable = inception_trainable.get_layer('mixed7')
last_output_trainable = last_layer_trainable.output

In [79]:
x = Conv2D(filters=16, kernel_size=(3, 3), padding="same" , activation="relu")(last_output_trainable)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=32, kernel_size=(3, 3), padding="same" , activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=(3, 3), padding="same" , activation="relu")(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(units = 128, activation = "relu")(x)
x = BatchNormalization()(x)

output_trainable = Dense(units = len(label_less_cols), activation = "sigmoid")(x)

base_inception_trainable = tf.keras.Model(inception_trainable.input, output_trainable)

base_inception_trainable.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy", Precision(), auc]
)

base_inception_trainable.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 712, 1072,   0           []                               
                                3)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 355, 535, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 355, 535, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 conv2d_7 (Conv2D)              (None, 86, 131, 64)  76800       ['activation_6[0][0]']           
                                                                                                  
 conv2d_10 (Conv2D)             (None, 86, 131, 96)  82944       ['activation_9[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 86, 131, 32)  6144        ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 86, 131, 64)  192        ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 86, 131, 64)  192        ['conv2d_7[0][0]']               
 rmalizati

 mixed1 (Concatenate)           (None, 86, 131, 288  0           ['activation_12[0][0]',          
                                )                                 'activation_14[0][0]',          
                                                                  'activation_17[0][0]',          
                                                                  'activation_18[0][0]']          
                                                                                                  
 conv2d_22 (Conv2D)             (None, 86, 131, 64)  18432       ['mixed1[0][0]']                 
                                                                                                  
 batch_normalization_22 (BatchN  (None, 86, 131, 64)  192        ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

 batch_normalization_29 (BatchN  (None, 42, 65, 96)  288         ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_26 (Activation)     (None, 42, 65, 384)  0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 42, 65, 96)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 42, 65, 288)  0          ['mixed2[0][0]']                 
                                                                                                  
 mixed3 (Concatenate)           (None, 42, 65, 768)  0           ['activation_26[0][0]',          
          

                                                                                                  
 activation_39 (Activation)     (None, 42, 65, 192)  0           ['batch_normalization_39[0][0]'] 
                                                                                                  
 mixed4 (Concatenate)           (None, 42, 65, 768)  0           ['activation_30[0][0]',          
                                                                  'activation_33[0][0]',          
                                                                  'activation_38[0][0]',          
                                                                  'activation_39[0][0]']          
                                                                                                  
 conv2d_44 (Conv2D)             (None, 42, 65, 160)  122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_nor

                                                                  'activation_48[0][0]',          
                                                                  'activation_49[0][0]']          
                                                                                                  
 conv2d_54 (Conv2D)             (None, 42, 65, 160)  122880      ['mixed5[0][0]']                 
                                                                                                  
 batch_normalization_54 (BatchN  (None, 42, 65, 160)  480        ['conv2d_54[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_54 (Activation)     (None, 42, 65, 160)  0           ['batch_normalization_54[0][0]'] 
                                                                                                  
 conv2d_55

 batch_normalization_64 (BatchN  (None, 42, 65, 192)  576        ['conv2d_64[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_64 (Activation)     (None, 42, 65, 192)  0           ['batch_normalization_64[0][0]'] 
                                                                                                  
 conv2d_65 (Conv2D)             (None, 42, 65, 192)  258048      ['activation_64[0][0]']          
                                                                                                  
 batch_normalization_65 (BatchN  (None, 42, 65, 192)  576        ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

 conv2d_95 (Conv2D)             (None, 21, 32, 32)   4640        ['max_pooling2d_4[0][0]']        
                                                                                                  
 batch_normalization_95 (BatchN  (None, 21, 32, 32)  128         ['conv2d_95[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 max_pooling2d_5 (MaxPooling2D)  (None, 10, 16, 32)  0           ['batch_normalization_95[0][0]'] 
                                                                                                  
 conv2d_96 (Conv2D)             (None, 10, 16, 64)   18496       ['max_pooling2d_5[0][0]']        
                                                                                                  
 batch_normalization_96 (BatchN  (None, 10, 16, 64)  256         ['conv2d_96[0][0]']              
 ormalizat

In [80]:
base_inception_trainable_history = base_inception_trainable.fit(
    train_less_gen,
    validation_data=val_less_gen,
    epochs = 1)

base_inception_trainable.evaluate(test_less_gen)

80/80 [==============================] - 451s 6s/step - loss: 0.5185 - accuracy: 0.1219 - precision: 0.0000e+00 - auc: 0.5000


[0.5184757113456726, 0.12187500298023224, 0.0, 0.5]

La capacidad añadida no rindio los frutos esperados